<a href="https://colab.research.google.com/github/Xiaoyan-Fu/amptorch-AL-specific_atoms/blob/active_learning/zeolite-adsorption-energy/ad_zeolite_amptorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/Xiaoyan-Fu/amptorch.git@active_learning
!pip install -q git+https://github.com/mshuaibii/SIMPLE-NN.git
!pip install -q amp-atomistics
!pip install -q --upgrade dftbook

     |████████████████████████████████| 296kB 2.8MB/s 
     |████████████████████████████████| 6.7MB 7.6MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 2.1MB 42.5MB/s 
     |████████████████████████████████| 12.8MB 338kB/s 
     |████████████████████████████████| 110.5MB 102kB/s 
     |████████████████████████████████| 512kB 47.1MB/s 
     |████████████████████████████████| 3.8MB 33.8MB/s 
  ERROR: Failed building wheel for simple-nn
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.5 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.5 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
    Running setup.py install for simple-nn ... do

In [0]:
from sklearn.metrics import mean_squared_error
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from ase.visualize import view
import ase.io
from amp import Amp
from amp.model.neuralnetwork import NeuralNetwork
from amp.model import LossFunction
import operator
import amptorch
import copy
import matplotlib
from skorch import NeuralNetRegressor
from skorch.dataset import CVSplit
from skorch.callbacks import Checkpoint, EpochScoring
from skorch.callbacks.lr_scheduler import LRScheduler
import skorch.callbacks.base
from amptorch.gaussian import SNN_Gaussian
from amptorch.model import BPNN, CustomMSELoss
from amptorch.data_preprocess import AtomsDataset, factorize_data, collate_amp, TestDataset
from amptorch.skorch_model import AMP
from amptorch.skorch_model.utils import target_extractor, energy_score, forces_score
from amptorch.analysis import parity_plot
from torch.utils.data import DataLoader
from torch.nn import init
from skorch.utils import to_numpy
import matplotlib.pyplot as plt

In [0]:
def Select_image(images):
    train_image = []
    test_image = []
    for index,image in enumerate(images[:]):
        if index-index//3*3 == 0:
            test_image.append(image)
        else:
            train_image.append(image)
    print(len(train_image), len(test_image))
    return train_image, test_image

In [0]:
def run_model(assignments, numberi=0):
    class train_end_load_best_valid_loss(skorch.callbacks.base.Callback):
        def on_train_end(self, net, X, y):
            net.load_params('valid_best_params.pt')
    out_file = open('log-sigopt','a')
    # LR_schedule = LRScheduler("CosineAnnealingLR", T_max=assignments["T_max"])
    cp = Checkpoint(monitor='valid_loss_best', fn_prefix='valid_best_')
    load_best_valid_loss = train_end_load_best_valid_loss()
    images = ase.io.read('./traj_taged_adsorptionenergy.traj', index=':')
    Gs = {}
    Gs["G2_etas"] = np.logspace(np.log10(0.05), np.log10(5.0), num=4)
    Gs["G2_rs_s"] = [0] * 4
    Gs["G4_etas"] = [0.005]
    Gs["G4_zetas"] = [1.0]
    Gs["G4_gammas"] = [+1.0, -1]
    Gs["cutoff"] = 6.5
    forcetraining = False
    label = "sigopt-zeolite"
    train_images, test_images = Select_image(images)

    DFT_energies_test = [image.get_potential_energy() for image in test_images]
    DFT_energies_train = [image.get_potential_energy() for image in train_images]
    training_data = AtomsDataset(train_images, SNN_Gaussian, Gs, forcetraining=forcetraining,
            label=label, cores=1, delta_data=None, specific_atoms=True)

    unique_atoms = training_data.elements
    fp_length = training_data.fp_length
    device = "cpu"
    # torch.set_num_threads(1)
    optimizer = optim.LBFGS
    batch_size = len(training_data)
    net = NeuralNetRegressor(
        module=BPNN(
            unique_atoms,
            [fp_length, assignments["hidden_layers"], assignments["num_nodes"]],
            device,
            forcetraining=forcetraining,
        ),
        criterion=CustomMSELoss,
        criterion__force_coefficient=0,
        optimizer=optimizer,
        #optimizer=torch.optim.LBFGS,
        lr=assignments["learning_rate"],
        #lr=1e-1,
        batch_size=batch_size,
        max_epochs=assignments["epochs"],
        iterator_train__collate_fn=collate_amp,
        iterator_train__shuffle=False,
        iterator_valid__collate_fn=collate_amp,
        iterator_valid__shuffle=False,
        device=device,
        train_split=CVSplit(cv=0.2),
        callbacks=[
            EpochScoring(
                energy_score,
                on_train=True,
                use_caching=True,
                target_extractor=target_extractor,
            ),
             cp,
             load_best_valid_loss,
            # LR_schedule
        ],
    )
    calc = AMP(training_data, net, label,specific_atoms=True)
    calc.train(overwrite=True)
    for image in test_images:
        image.set_calculator(calc)
    for image in train_images:
        image.set_calculator(calc)
    pred_energies_test = [image.get_potential_energy() for image in test_images]
    pred_energies_train = [image.get_potential_energy() for image in train_images]
    for e_t in pred_energies_test:
        if np.isnan(e_t):
            e_t = 1e10
    for e_tr in pred_energies_train:
        if np.isnan(e_tr):
            e_tr =1e10
    energy_rmse_test = mean_squared_error(pred_energies_test, DFT_energies_test)
    energy_rmse_train = mean_squared_error(pred_energies_train, DFT_energies_train)
    print('***************', file=out_file)
    print('No. ', numberi, file=out_file)
    print('**RMSE_TEST:', energy_rmse_test,file=out_file)
    print('**RMSE_Train:', energy_rmse_train,file=out_file)
    print('Energies:::::::', file=out_file)
    print('train data',file=out_file)
    for i in range(len(train_images)):
        print('(',DFT_energies_train[i],';',pred_energies_train[i],') ', end='',file=out_file)
    print('****',file=out_file)
    print('test data', file=out_file)
    for i in range(len(test_images)):
        print('(',DFT_energies_test[i],';',pred_energies_test[i],') ', end='',file=out_file)
    print('****',file=out_file)
    fig = plt.figure('ML vs DFT')
    ax = fig.add_subplot(111)
    ax.scatter(DFT_energies_train, pred_energies_train, c='b')
    ax.scatter(DFT_energies_test, pred_energies_test, c='r')
    ax.set_xlabel('E_DFT')
    ax.set_ylabel('E_ML')
    figure_name = './DFTvsML'+ str(numberi)+ '.png'
    fig.savefig(figure_name, bbox_inches='tight')
    fig = 0
    return energy_rmse_test, energy_rmse_train

In [0]:
assignments = {
        'epochs': 995,
        'learning_rate': 0.02315667835660472,
        'hidden_layers': 7,
        'num_nodes': 36
        }

In [16]:
run_model(assignments)

720 361
Calculating fingerprints...
Fingerprints Calculated!
  epoch    energy_score    train_loss    valid_loss    cp     dur
-------  --------------  ------------  ------------  ----  ------
      1          0.4960      224.5067      150.7501     +  3.9473
      2          0.4448      180.5567      152.2698        3.6482
      3          0.4060      150.4346      149.7562     +  3.6330
      4          0.3715      125.9148      142.4012     +  3.7367
      5          0.3373      103.7800      136.3329     +  3.8739
      6          0.3088       87.0196      129.5833     +  3.9035
      7          0.2623       62.7981      125.5408     +  3.8898
      8          0.2264       46.7551      123.9199     +  3.9081
      9          0.2090       39.8498      123.1378     +  3.9931
     10          0.1922       33.7035      123.9729        4.0229
     11          0.1778       28.8496      126.4363        5.5012
     12          0.1642       24.6103      127.3127        6.3160
     13        

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


(5.498453340761248, 1.4810848891519681)